In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm
import re
import time
import requests
from bs4 import BeautifulSoup

driver = webdriver.Chrome("C:/Users/user/Documents/team/chromedriver/chromedriver.exe")

comm_list = []
date_list = []

page_num = 9999
goods_no = 26847  # 바꿔주세요

base = 'https://www.kurly.com/shop/goods/goods_review_list.php?goodsno='+str(goods_no)+'&page='+str(page_num)
driver.get(base)
time.sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
last= soup.select('#contents-wrapper > div.board_pg_area > strong')
last_page = last[0].text

time.sleep(1)

for page in tqdm(range(1,int(last_page)+1)):
    base = 'https://www.kurly.com/shop/goods/goods_review_list.php?goodsno='+str(goods_no)+'&page='+str(page)
    driver.get(base)
    time.sleep(1.5)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    review_list = re.findall(
        '<p class="package">.+?<div class="review_photo"></div>(.+?)</div>.+?<div class="goods-review-grp-btn">', html,
        re.DOTALL)
    for row in review_list:
        comm_list.append(row)
        
    rev = soup.select('div.inner_review')
    date = soup.select('td.time')
    for i,x in enumerate(rev):
        date_list.append(date[i].text)

time.sleep(5)
driver.close()

In [6]:
import pandas as pd 
df = pd.DataFrame({'comment':comm_list[2:], 'date':date_list[2:]})

df['date'] = df['date'].astype(str)
df['date'] =  pd.to_datetime(df['date'], format='%Y/%m/%d') #시간형식 바꾸기 

df['comment'] = df['comment'].str.replace("[^ 가-힣A-Za-z]", "")
df['comment'] = df['comment'].str.replace("br", "")

#df.to_csv("hodu_review_kurly.csv", index=False, encoding="utf-8-sig") # Name Change
df

,comment,date
0,학화호두 생각했는데 그맛과는 살짝 다른듯담번엔 팥으로 사렵니다,2020-12-18
1,백앙금의매력마의 약간의 미끄러우하는 느낌도 있고 밋있어요,2020-12-18
2,맛있어요 달지않고,2020-12-16
3,에어프라이에 돌려먹음 고소하니 맛있어요,2020-12-15
4,백앙금샀는데 앙금이좀 미끌거려요,2020-12-13
...,...,...
2804,백앙금 호두과자 먹기싶어서 주문했어요,2020-03-19
2805,호두과자 좋아하는데그냥그냥 그래요재구매는 안할거 같아요,2020-03-19
2806,안동참마호두과자 첫 구매에요 양이 좋긴한데 맛있네요 부드럽고,2020-03-19
2807,상품 설명 처럼 많이 달지 않아서 좋고마 특유의 진득함이 약간씩 느껴집니다 하지만 ...,2020-03-19
